In [1]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

from datetime import datetime

from sklearn import preprocessing
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.model_selection import learning_curve, GridSearchCV, train_test_split

# Preprocess Steps

In [2]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
# Load Train dataset
test_1 = pd.read_csv("../data/201127_Cycles/data_120sec_overlap_10parts/test1.csv", delimiter = ";")
test_2 = pd.read_csv("../data/201127_Cycles/data_120sec_overlap_10parts/test2.csv", delimiter = ";")

In [4]:
test_1.head()

,Cycle,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,Test1,3.918951,13.532863,0,5.298202,2.830860,-3.379077,0.691492,0.861162,0.803558,-0.806347,0.215596,0.165138,0.555046,3,40.333333,1.552511,11
1,Test1,4.221303,14.551459,0,5.087991,1.981586,-3.960704,0.560160,0.879376,0.708638,-0.970036,0.293578,0.215596,0.458716,5,20.000000,1.561644,14
2,Test1,5.452417,14.986454,0,5.294224,2.328557,-3.960704,0.609679,0.899586,0.825244,-1.021126,0.357798,0.243119,0.348624,6,12.666667,1.694064,21
3,Test1,6.843855,14.986454,0,4.739604,2.481710,-3.960704,0.624350,0.863683,0.784511,-0.895674,0.353211,0.325688,0.229358,3,16.666667,1.808219,21
4,Test1,6.925873,12.343750,0,3.508140,2.481710,-2.303663,0.435310,0.503358,0.568196,-0.616491,0.376147,0.412844,0.077982,2,8.500000,1.726027,19


In [5]:
test_2.head()

,Cycle,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,Test2,6.465940,12.916667,0.0,4.260583,3.259251,-2.899711,0.592759,0.683725,0.643375,-0.672484,0.382759,0.306897,0.165517,6,8.000000,1.783505,20
1,Test2,7.904128,17.077014,0.0,4.791776,2.299987,-2.754001,0.440338,0.721065,0.609165,-0.773342,0.393103,0.289655,0.144828,3,14.000000,1.969072,21
2,Test2,10.160489,20.276449,0.0,6.874336,3.105919,-3.039034,0.564462,0.758665,0.715666,-0.868041,0.368966,0.282759,0.213793,3,20.666667,2.274914,19
3,Test2,11.642128,20.276449,0.0,6.240755,3.105919,-3.039034,0.510124,0.497817,0.523712,-0.518775,0.351724,0.306897,0.093103,3,9.000000,2.474227,14
4,Test2,9.163712,16.847016,0.0,6.020022,1.758849,-1.701132,0.327585,0.382483,0.432201,-0.426929,0.255172,0.275862,0.182759,2,26.500000,2.213058,12


In [6]:
# Load Train dataset
cycle_df = pd.read_csv("../data/trainData.csv", delimiter = ";")

In [7]:
cycle_df.head()

,Cycle,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,1,2.227640,10.237216,0.0,2.569309,2.503424,-2.056384,0.537237,0.551614,0.889754,-0.752517,0.229630,0.266667,0.429630,2,29.000000,1.095588,6
1,1,2.481401,10.237216,0.0,2.614335,2.503424,-2.056384,0.562851,0.550084,0.896373,-0.724675,0.288889,0.281481,0.340741,3,15.333333,1.117647,7
2,1,3.533260,11.265408,0.0,3.291818,2.503424,-2.056384,0.551675,0.542847,0.795549,-0.692367,0.362963,0.303704,0.229630,3,10.333333,1.242647,9
3,1,4.412876,11.265408,0.0,3.338501,2.503424,-2.056384,0.551977,0.525758,0.784108,-0.665086,0.370370,0.362963,0.118519,3,5.333333,1.352941,9
4,1,4.304404,11.265408,0.0,3.439727,2.682240,-2.056384,0.638003,0.514284,0.836900,-0.686816,0.318519,0.407407,0.118519,3,5.333333,1.367647,9


In [8]:
test_1.head(5).append(test_1.tail(5))

,Cycle,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,Test1,3.918951,13.532863,0,5.298202,2.830860,-3.379077,0.691492,0.861162,0.803558,-0.806347,0.215596,0.165138,0.555046,3,40.333333,1.552511,11
1,Test1,4.221303,14.551459,0,5.087991,1.981586,-3.960704,0.560160,0.879376,0.708638,-0.970036,0.293578,0.215596,0.458716,5,20.000000,1.561644,14
2,Test1,5.452417,14.986454,0,5.294224,2.328557,-3.960704,0.609679,0.899586,0.825244,-1.021126,0.357798,0.243119,0.348624,6,12.666667,1.694064,21
3,Test1,6.843855,14.986454,0,4.739604,2.481710,-3.960704,0.624350,0.863683,0.784511,-0.895674,0.353211,0.325688,0.229358,3,16.666667,1.808219,21
4,Test1,6.925873,12.343750,0,3.508140,2.481710,-2.303663,0.435310,0.503358,0.568196,-0.616491,0.376147,0.412844,0.077982,2,8.500000,1.726027,19
5,Test1,4.227522,10.329861,0,3.479947,1.767000,-2.059642,0.393284,0.436261,0.508913,-0.570390,0.321101,0.362385,0.215596,4,11.750000,1.392694,11
6,Test1,2.221494,8.038194,0,2.233897,1.371213,-1.974824,0.271681,0.362659,0.418563,-0.438957,0.348624,0.366972,0.197248,6,7.166667,1.132420,6
7,Test1,1.636098,4.268667,0,1.305546,1.174169,-1.013865,0.250637,0.199778,0.389065,-0.319122,0.316514,0.380734,0.206422,6,7.500000,1.000000,0
8,Test1,1.858510,7.322273,0,1.644773,1.487938,-3.547100,0.343417,0.457008,0.417379,-0.375993,0.302752,0.330275,0.288991,5,12.600000,1.027397,2
9,Test1,2.720898,11.132484,0,3.041469,1.487938,-3.547100,0.316660,0.661484,0.525432,-0.674312,0.344037,0.279817,0.307339,7,9.571429,1.168950,8


In [9]:
test_2.head(5).append(test_2.tail(5))

,Cycle,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,Test2,6.465940,12.916667,0.0000,4.260583,3.259251,-2.899711,0.592759,0.683725,0.643375,-0.672484,0.382759,0.306897,0.165517,6,8.000000,1.783505,20
1,Test2,7.904128,17.077014,0.0000,4.791776,2.299987,-2.754001,0.440338,0.721065,0.609165,-0.773342,0.393103,0.289655,0.144828,3,14.000000,1.969072,21
2,Test2,10.160489,20.276449,0.0000,6.874336,3.105919,-3.039034,0.564462,0.758665,0.715666,-0.868041,0.368966,0.282759,0.213793,3,20.666667,2.274914,19
3,Test2,11.642128,20.276449,0.0000,6.240755,3.105919,-3.039034,0.510124,0.497817,0.523712,-0.518775,0.351724,0.306897,0.093103,3,9.000000,2.474227,14
4,Test2,9.163712,16.847016,0.0000,6.020022,1.758849,-1.701132,0.327585,0.382483,0.432201,-0.426929,0.255172,0.275862,0.182759,2,26.500000,2.213058,12
5,Test2,8.901176,16.847016,0.0000,6.326164,2.121295,-2.935398,0.471438,0.715200,0.642897,-0.820790,0.306897,0.241379,0.227586,3,22.000000,2.158076,16
6,Test2,10.588989,26.186539,0.0000,8.734744,3.052000,-2.935398,0.722320,0.752837,0.709482,-0.807740,0.344828,0.262069,0.255172,4,18.500000,2.415808,22
7,Test2,21.135704,26.186539,0.0000,4.736405,3.052000,-1.018415,0.623968,0.150260,0.519474,-0.219485,0.268966,0.286207,0.020690,1,6.000000,3.838488,11
8,Test2,22.990274,25.410812,19.9421,0.869349,0.902967,-0.231545,0.201926,0.038700,0.276874,-0.157071,0.113793,0.124138,0.000000,0,0.000000,4.024055,2
9,Test2,20.076108,24.126247,0.0000,5.212958,1.432352,-1.913952,0.334881,0.472691,0.379482,-0.551738,0.151724,0.241379,0.003448,1,1.000000,3.646048,7


In [10]:
test_1.shape, test_2.shape, cycle_df.shape

((10, 18), (10, 18), (360, 18))

In [11]:
cycle_df.columns

Index(['Cycle', 'meanVel', 'maxVel', 'minVel', 'stdVel', 'maxAccel', 'maxDecel', 'stdAccel', 'stdDecel', 'meanAccel', 'meanDecel', 'rAccel', 'rDecel', 'rStop', 'stopCnt', 'meanStopDur', 'meanGear', 'gearChg'], dtype='object')

In [12]:
determine_columns = ['meanVel', 'maxVel', 'minVel', 'stdVel', 'maxAccel',
       'maxDecel', 'stdAccel', 'stdDecel', 'meanAccel', 'meanDecel', 'rAccel',
       'rDecel', 'rStop', 'stopCnt', 'meanStopDur', 'meanGear', 'gearChg']

In [13]:
cycle_df.head()

,Cycle,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,1,2.227640,10.237216,0.0,2.569309,2.503424,-2.056384,0.537237,0.551614,0.889754,-0.752517,0.229630,0.266667,0.429630,2,29.000000,1.095588,6
1,1,2.481401,10.237216,0.0,2.614335,2.503424,-2.056384,0.562851,0.550084,0.896373,-0.724675,0.288889,0.281481,0.340741,3,15.333333,1.117647,7
2,1,3.533260,11.265408,0.0,3.291818,2.503424,-2.056384,0.551675,0.542847,0.795549,-0.692367,0.362963,0.303704,0.229630,3,10.333333,1.242647,9
3,1,4.412876,11.265408,0.0,3.338501,2.503424,-2.056384,0.551977,0.525758,0.784108,-0.665086,0.370370,0.362963,0.118519,3,5.333333,1.352941,9
4,1,4.304404,11.265408,0.0,3.439727,2.682240,-2.056384,0.638003,0.514284,0.836900,-0.686816,0.318519,0.407407,0.118519,3,5.333333,1.367647,9


# Prediction Steps

In [14]:
cycle_df.iloc[:,0]

0       1
1       1
2       1
3       1
4       1
       ..
355    12
356    12
357    12
358    12
359    12
Name: Cycle, Length: 360, dtype: int64

In [16]:
cycle_df.iloc[:,1:18]

,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg
0,2.227640,10.237216,0.000000,2.569309,2.503424,-2.056384,0.537237,0.551614,0.889754,-0.752517,0.229630,0.266667,0.429630,2,29.000000,1.095588,6
1,2.481401,10.237216,0.000000,2.614335,2.503424,-2.056384,0.562851,0.550084,0.896373,-0.724675,0.288889,0.281481,0.340741,3,15.333333,1.117647,7
2,3.533260,11.265408,0.000000,3.291818,2.503424,-2.056384,0.551675,0.542847,0.795549,-0.692367,0.362963,0.303704,0.229630,3,10.333333,1.242647,9
3,4.412876,11.265408,0.000000,3.338501,2.503424,-2.056384,0.551977,0.525758,0.784108,-0.665086,0.370370,0.362963,0.118519,3,5.333333,1.352941,9
4,4.304404,11.265408,0.000000,3.439727,2.682240,-2.056384,0.638003,0.514284,0.836900,-0.686816,0.318519,0.407407,0.118519,3,5.333333,1.367647,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,33.331393,37.347222,29.594444,2.096329,0.900000,-0.844444,0.207025,0.228630,0.384184,-0.367718,0.392000,0.296000,0.000000,0,0.000000,5.000000,0
356,33.440961,37.347222,29.594444,2.012819,0.900000,-0.936111,0.209060,0.243186,0.383044,-0.404696,0.384000,0.336000,0.000000,0,0.000000,5.000000,0
357,33.377006,37.347222,27.827778,2.124085,0.794444,-0.936111,0.185990,0.254159,0.359785,-0.439953,0.352000,0.376000,0.000000,0,0.000000,5.000000,0
358,33.050287,37.347222,22.875000,2.671342,0.794444,-1.586111,0.186108,0.339817,0.365447,-0.511111,0.328000,0.416000,0.000000,0,0.000000,4.984127,1


In [17]:
X = cycle_df.iloc[:,1:18]
y = cycle_df.iloc[:,0]

In [21]:
test_1 = test_1.drop(columns='Cycle')
test_2 = test_2.drop(columns='Cycle')

In [22]:
X.columns

Index(['meanVel', 'maxVel', 'minVel', 'stdVel', 'maxAccel', 'maxDecel', 'stdAccel', 'stdDecel', 'meanAccel', 'meanDecel', 'rAccel', 'rDecel', 'rStop', 'stopCnt', 'meanStopDur', 'meanGear', 'gearChg'], dtype='object')

In [23]:
test_1.columns

Index(['meanVel', 'maxVel', 'minVel', 'stdVel', 'maxAccel', 'maxDecel', 'stdAccel', 'stdDecel', 'meanAccel', 'meanDecel', 'rAccel', 'rDecel', 'rStop', 'stopCnt', 'meanStopDur', 'meanGear', 'gearChg'], dtype='object')

In [24]:
test_2.columns

Index(['meanVel', 'maxVel', 'minVel', 'stdVel', 'maxAccel', 'maxDecel', 'stdAccel', 'stdDecel', 'meanAccel', 'meanDecel', 'rAccel', 'rDecel', 'rStop', 'stopCnt', 'meanStopDur', 'meanGear', 'gearChg'], dtype='object')

In [25]:
# load the model from disk
filename = '../models/KNC_model_120_30.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [29]:
best_KNC = loaded_model.fit(X, y)
phase_predictor_1 = best_KNC.predict(test_1)
phase_predictor_2 = best_KNC.predict(test_2)

print(phase_predictor_1)

[2 7 7 7 8 1 7 7 1 1]


In [30]:
print(phase_predictor_2)

[ 8  7  3  3 11  7  9 10  6  3]


In [32]:
test_1['phase_prediction'] = phase_predictor_1
test_2['phase_prediction'] = phase_predictor_2


In [35]:
test_1.head(10)

,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg,phase_prediction
0,3.918951,13.532863,0,5.298202,2.830860,-3.379077,0.691492,0.861162,0.803558,-0.806347,0.215596,0.165138,0.555046,3,40.333333,1.552511,11,2
1,4.221303,14.551459,0,5.087991,1.981586,-3.960704,0.560160,0.879376,0.708638,-0.970036,0.293578,0.215596,0.458716,5,20.000000,1.561644,14,7
2,5.452417,14.986454,0,5.294224,2.328557,-3.960704,0.609679,0.899586,0.825244,-1.021126,0.357798,0.243119,0.348624,6,12.666667,1.694064,21,7
3,6.843855,14.986454,0,4.739604,2.481710,-3.960704,0.624350,0.863683,0.784511,-0.895674,0.353211,0.325688,0.229358,3,16.666667,1.808219,21,7
4,6.925873,12.343750,0,3.508140,2.481710,-2.303663,0.435310,0.503358,0.568196,-0.616491,0.376147,0.412844,0.077982,2,8.500000,1.726027,19,8
5,4.227522,10.329861,0,3.479947,1.767000,-2.059642,0.393284,0.436261,0.508913,-0.570390,0.321101,0.362385,0.215596,4,11.750000,1.392694,11,1
6,2.221494,8.038194,0,2.233897,1.371213,-1.974824,0.271681,0.362659,0.418563,-0.438957,0.348624,0.366972,0.197248,6,7.166667,1.132420,6,7
7,1.636098,4.268667,0,1.305546,1.174169,-1.013865,0.250637,0.199778,0.389065,-0.319122,0.316514,0.380734,0.206422,6,7.500000,1.000000,0,7
8,1.858510,7.322273,0,1.644773,1.487938,-3.547100,0.343417,0.457008,0.417379,-0.375993,0.302752,0.330275,0.288991,5,12.600000,1.027397,2,1
9,2.720898,11.132484,0,3.041469,1.487938,-3.547100,0.316660,0.661484,0.525432,-0.674312,0.344037,0.279817,0.307339,7,9.571429,1.168950,8,1


In [36]:
test_2.head(10)

,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg,phase_prediction
0,6.465940,12.916667,0.0000,4.260583,3.259251,-2.899711,0.592759,0.683725,0.643375,-0.672484,0.382759,0.306897,0.165517,6,8.000000,1.783505,20,8
1,7.904128,17.077014,0.0000,4.791776,2.299987,-2.754001,0.440338,0.721065,0.609165,-0.773342,0.393103,0.289655,0.144828,3,14.000000,1.969072,21,7
2,10.160489,20.276449,0.0000,6.874336,3.105919,-3.039034,0.564462,0.758665,0.715666,-0.868041,0.368966,0.282759,0.213793,3,20.666667,2.274914,19,3
3,11.642128,20.276449,0.0000,6.240755,3.105919,-3.039034,0.510124,0.497817,0.523712,-0.518775,0.351724,0.306897,0.093103,3,9.000000,2.474227,14,3
4,9.163712,16.847016,0.0000,6.020022,1.758849,-1.701132,0.327585,0.382483,0.432201,-0.426929,0.255172,0.275862,0.182759,2,26.500000,2.213058,12,11
5,8.901176,16.847016,0.0000,6.326164,2.121295,-2.935398,0.471438,0.715200,0.642897,-0.820790,0.306897,0.241379,0.227586,3,22.000000,2.158076,16,7
6,10.588989,26.186539,0.0000,8.734744,3.052000,-2.935398,0.722320,0.752837,0.709482,-0.807740,0.344828,0.262069,0.255172,4,18.500000,2.415808,22,9
7,21.135704,26.186539,0.0000,4.736405,3.052000,-1.018415,0.623968,0.150260,0.519474,-0.219485,0.268966,0.286207,0.020690,1,6.000000,3.838488,11,10
8,22.990274,25.410812,19.9421,0.869349,0.902967,-0.231545,0.201926,0.038700,0.276874,-0.157071,0.113793,0.124138,0.000000,0,0.000000,4.024055,2,6
9,20.076108,24.126247,0.0000,5.212958,1.432352,-1.913952,0.334881,0.472691,0.379482,-0.551738,0.151724,0.241379,0.003448,1,1.000000,3.646048,7,3


In [37]:
#Export dataframe as a csv
test_1.to_csv("../data/results/test_1_result.csv", index=0)
test_2.to_csv("../data/results/test_2_result.csv", index=0)

In [38]:
test_1.phase_prediction.keys

<bound method Series.keys of 0    2
1    7
2    7
3    7
4    8
5    1
6    7
7    7
8    1
9    1
Name: phase_prediction, dtype: int64>

In [39]:
test_2.phase_prediction.keys

<bound method Series.keys of 0     8
1     7
2     3
3     3
4    11
5     7
6     9
7    10
8     6
9     3
Name: phase_prediction, dtype: int64>

In [40]:
# Load cycle_df dataset
test1_results = pd.read_csv("../data/results/test_1_result.csv")
test2_results = pd.read_csv("../data/results/test_2_result.csv")

In [42]:
test_1.head(20)

,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg,phase_prediction
0,3.918951,13.532863,0,5.298202,2.830860,-3.379077,0.691492,0.861162,0.803558,-0.806347,0.215596,0.165138,0.555046,3,40.333333,1.552511,11,2
1,4.221303,14.551459,0,5.087991,1.981586,-3.960704,0.560160,0.879376,0.708638,-0.970036,0.293578,0.215596,0.458716,5,20.000000,1.561644,14,7
2,5.452417,14.986454,0,5.294224,2.328557,-3.960704,0.609679,0.899586,0.825244,-1.021126,0.357798,0.243119,0.348624,6,12.666667,1.694064,21,7
3,6.843855,14.986454,0,4.739604,2.481710,-3.960704,0.624350,0.863683,0.784511,-0.895674,0.353211,0.325688,0.229358,3,16.666667,1.808219,21,7
4,6.925873,12.343750,0,3.508140,2.481710,-2.303663,0.435310,0.503358,0.568196,-0.616491,0.376147,0.412844,0.077982,2,8.500000,1.726027,19,8
5,4.227522,10.329861,0,3.479947,1.767000,-2.059642,0.393284,0.436261,0.508913,-0.570390,0.321101,0.362385,0.215596,4,11.750000,1.392694,11,1
6,2.221494,8.038194,0,2.233897,1.371213,-1.974824,0.271681,0.362659,0.418563,-0.438957,0.348624,0.366972,0.197248,6,7.166667,1.132420,6,7
7,1.636098,4.268667,0,1.305546,1.174169,-1.013865,0.250637,0.199778,0.389065,-0.319122,0.316514,0.380734,0.206422,6,7.500000,1.000000,0,7
8,1.858510,7.322273,0,1.644773,1.487938,-3.547100,0.343417,0.457008,0.417379,-0.375993,0.302752,0.330275,0.288991,5,12.600000,1.027397,2,1
9,2.720898,11.132484,0,3.041469,1.487938,-3.547100,0.316660,0.661484,0.525432,-0.674312,0.344037,0.279817,0.307339,7,9.571429,1.168950,8,1


In [43]:

test1_results.head(20)

,meanVel,maxVel,minVel,stdVel,maxAccel,maxDecel,stdAccel,stdDecel,meanAccel,meanDecel,rAccel,rDecel,rStop,stopCnt,meanStopDur,meanGear,gearChg,phase_prediction
0,3.918951,13.532863,0,5.298202,2.830860,-3.379077,0.691492,0.861162,0.803558,-0.806347,0.215596,0.165138,0.555046,3,40.333333,1.552511,11,2
1,4.221303,14.551459,0,5.087991,1.981586,-3.960704,0.560160,0.879376,0.708638,-0.970036,0.293578,0.215596,0.458716,5,20.000000,1.561644,14,7
2,5.452417,14.986454,0,5.294224,2.328557,-3.960704,0.609679,0.899586,0.825244,-1.021126,0.357798,0.243119,0.348624,6,12.666667,1.694064,21,7
3,6.843855,14.986454,0,4.739604,2.481710,-3.960704,0.624350,0.863683,0.784511,-0.895674,0.353211,0.325688,0.229358,3,16.666667,1.808219,21,7
4,6.925873,12.343750,0,3.508140,2.481710,-2.303663,0.435310,0.503358,0.568196,-0.616491,0.376147,0.412844,0.077982,2,8.500000,1.726027,19,8
5,4.227522,10.329861,0,3.479947,1.767000,-2.059642,0.393284,0.436261,0.508913,-0.570390,0.321101,0.362385,0.215596,4,11.750000,1.392694,11,1
6,2.221494,8.038194,0,2.233897,1.371213,-1.974824,0.271681,0.362659,0.418563,-0.438957,0.348624,0.366972,0.197248,6,7.166667,1.132420,6,7
7,1.636098,4.268667,0,1.305546,1.174169,-1.013865,0.250637,0.199778,0.389065,-0.319122,0.316514,0.380734,0.206422,6,7.500000,1.000000,0,7
8,1.858510,7.322273,0,1.644773,1.487938,-3.547100,0.343417,0.457008,0.417379,-0.375993,0.302752,0.330275,0.288991,5,12.600000,1.027397,2,1
9,2.720898,11.132484,0,3.041469,1.487938,-3.547100,0.316660,0.661484,0.525432,-0.674312,0.344037,0.279817,0.307339,7,9.571429,1.168950,8,1
